In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from sklearn.model_selection import train_test_split
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *
from pyspark.mllib.util import MLUtils
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StringIndexer, OneHotEncoder
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, \
    auc

In [2]:
spark=SparkSession.builder.appName('Australian_rain_prediction').getOrCreate()
sc = spark.sparkContext

22/11/26 17:05:18 WARN Utils: Your hostname, bigdata2022-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/11/26 17:05:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/26 17:05:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark.read.csv('hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv',inferSchema=True,header=True)
#df.cache()

In [4]:
#indexer = StringIndexer(inputCol=None, ouputCols=None).fit(df)

In [5]:
print(df.columns)

['_c0', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'RainToday', 'RainTomorrow', 'Month', 'WindGustDir_N', 'WindGustDir_S', 'WindGustDir_W', 'WindDir3pm_N', 'WindDir3pm_S', 'WindDir3pm_W']


In [6]:
df_1 = df.drop('RainTomorrow')

In [7]:
assembler = VectorAssembler(inputCols=df_1.columns, outputCol="features_to_scale")

In [8]:
scaler = MinMaxScaler(inputCol="features_to_scale", outputCol="features_scaled")

In [9]:
#final_assembler = VectorAssembler(inputCols="features_scaled",outputCol="features")

In [10]:
model = GBTClassifier(labelCol="RainTomorrow", featuresCol="features_scaled", maxIter=10) 
#model = GradientBoostedTrees.trainClassifier(X_train_labled, categoricalFeaturesInfo={}, numIterations=3)

In [11]:
# Inizializing pipeline
# stages = indexer
# stages.append(assembler)
# stages.append(scaler)
# stages.append(model)
# pipeline = Pipeline().setStages(stages)
pipeline = Pipeline(stages=[assembler, scaler, model])

In [12]:
training_data, test_data = df.randomSplit([0.75, 0.25])

In [13]:
#training_data = df.drop('RainTomorrow')

In [14]:
#test_data = df.drop('RainTomorrow')

In [15]:
pipeline_model = pipeline.fit(training_data)

22/11/26 17:05:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/11/26 17:05:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
 Schema: _c0, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv
22/11/26 17:05:35 WARN CSVHeaderChecker: CSV header do

In [16]:
predictions = pipeline_model.transform(test_data)
#predictionAndLabels = test_data.map(lambda lp: (float(model.predict(lp.features)), lp.label))

In [17]:
predictions.select("prediction", "RainTomorrow").show(100)

22/11/26 17:05:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
 Schema: _c0, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv


+----------+------------+
|prediction|RainTomorrow|
+----------+------------+
|       0.0|           0|
|       0.0|           0|
|       0.0|           1|
|       1.0|           1|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           1|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           1|
|       1.0|           1|
|       1.0|           1|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|           0|
|       0.0|

22/11/26 17:05:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/26 17:05:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [18]:
t_test=predictions.select('RainTomorrow').toPandas()
t_hat=predictions.select('prediction').toPandas()

22/11/26 17:05:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
 Schema: _c0, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, RainToday, RainTomorrow, Month, WindGustDir_N, WindGustDir_S, WindGustDir_W, WindDir3pm_N, WindDir3pm_S, WindDir3pm_W
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv
22/11/26 17:05:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressu

In [20]:
print("\nAccuracy score on the test set: ", accuracy_score(t_test, t_hat))


Accuracy score on the test set:  0.8619049490939109
